In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Netflix Trends Analyzer 📊\n",
    "Comprehensive Exploratory Data Analysis (EDA) on Netflix Titles Dataset\n",
    "\n",
    "## Key Questions to Explore:\n",
    "1. 📈 How has Netflix's content grown over the years?\n",
    "2. 🌍 Which countries contribute most content?\n",
    "3. 🎭 Top 10 most frequent genres?\n",
    "4. 🎬 Who are the most featured actors/directors?\n",
    "5. 🕒 How long is the average content (duration)?\n",
    "6. 📅 Is there a trend in content addition by month/year?\n",
    "7. 🔞 What are the most common content ratings?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from collections import Counter\n",
    "import os\n",
    "from datetime import datetime\n",
    "\n",
    "# Set plot style and figure size\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "plt.rcParams['figure.figsize'] = (12, 8)\n",
    "plt.rcParams['font.size'] = 12\n",
    "\n",
    "# Create visuals directory if it doesn't exist\n",
    "os.makedirs('visuals', exist_ok=True)\n",
    "\n",
    "print(\"✅ Libraries imported successfully!\")\n",
    "print(f\"📁 Visuals will be saved to: {os.path.abspath('visuals')}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load and Explore Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load the dataset\n",
    "df = pd.read_csv('netflix_titles.csv')\n",
    "\n",
    "print(\"📊 Dataset Overview:\")\n",
    "print(f\"   • Total records: {len(df):,}\")\n",
    "print(f\"   • Columns: {len(df.columns)}\")\n",
    "print(f\"   • Date range: {df['release_year'].min()} - {df['release_year'].max()}\")\n",
    "\n",
    "print(\"\\n📋 Column Information:\")\n",
    "print(df.info())\n",
    "\n",
    "print(\"\\n🔍 First 5 rows:\")\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Cleaning & Preprocessing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check for missing values\n",
    "print(\"🔍 Missing Values Analysis:\")\n",
    "missing_data = df.isnull().sum()\n",
    "missing_percent = (missing_data / len(df)) * 100\n",
    "missing_df = pd.DataFrame({\n",
    "    'Missing Values': missing_data,\n",
    "    'Percentage': missing_percent\n",
    "})\n",
    "print(missing_df[missing_df['Missing Values'] > 0])\n",
    "\n",
    "# Convert 'date_added' to datetime\n",
    "df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')\n",
    "\n",
    "# Fill missing values\n",
    "df['country'] = df['country'].fillna('Unknown')\n",
    "df['director'] = df['director'].fillna('Unknown')\n",
    "df['cast'] = df['cast'].fillna('Unknown')\n",
    "df['rating'] = df['rating'].fillna('Unknown')\n",
    "\n",
    "# Extract year and month from date_added\n",
    "df['year_added'] = df['date_added'].dt.year\n",
    "df['month_added'] = df['date_added'].dt.month\n",
    "\n",
    "print(\"\\n✅ Data cleaning completed!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. 📈 Content Growth Over Years"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze content growth by year\n",
    "content_per_year = df.groupby('year_added').size().reset_index(name='count')\n",
    "content_per_year = content_per_year[content_per_year['year_added'].notna()]\n",
    "\n",
    "# Create visualization\n",
    "plt.figure(figsize=(14, 8))\n",
    "plt.plot(content_per_year['year_added'], content_per_year['count'], \n",
    "         marker='o', linewidth=3, markersize=8)\n",
    "plt.fill_between(content_per_year['year_added'], content_per_year['count'], \n",
    "                 alpha=0.3)\n",
    "plt.title('Netflix Content Growth Over Years', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Year Added to Netflix', fontsize=12)\n",
    "plt.ylabel('Number of Titles Added', fontsize=12)\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# Add annotations for key insights\n",
    "max_year = content_per_year.loc[content_per_year['count'].idxmax()]\n",
    "plt.annotate(f'Peak: {max_year[\"count\"]} titles in {int(max_year[\"year_added\"])}',\n",
    "             xy=(max_year['year_added'], max_year['count']),\n",
    "             xytext=(max_year['year_added']-2, max_year['count']+50),\n",
    "             arrowprops=dict(arrowstyle='->', color='red'),\n",
    "             fontsize=10, color='red')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('visuals/content_growth.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "# Print insights\n",
    "print(f\"📈 Key Insights:\")\n",
    "print(f\"   • Peak year: {int(max_year['year_added'])} with {max_year['count']} titles\")\n",
    "print(f\"   • Total growth: {content_per_year['count'].sum():,} titles\")\n",
    "print(f\"   • Average per year: {content_per_year['count'].mean():.0f} titles\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. 🌍 Top Countries by Content Count"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze countries\n",
    "country_counts = Counter()\n",
    "df['country'].dropna().apply(lambda x: country_counts.update([c.strip() for c in x.split(',')]))\n",
    "top_countries = country_counts.most_common(15)\n",
    "\n",
    "# Create visualization\n",
    "countries, counts = zip(*top_countries)\n",
    "plt.figure(figsize=(12, 10))\n",
    "bars = plt.barh(range(len(countries)), counts, color=sns.color_palette('viridis', len(countries)))\n",
    "plt.yticks(range(len(countries)), countries)\n",
    "plt.xlabel('Number of Titles', fontsize=12)\n",
    "plt.title('Top 15 Countries by Netflix Content Count', fontsize=16, fontweight='bold')\n",
    "\n",
    "# Add value labels on bars\n",
    "for i, (bar, count) in enumerate(zip(bars, counts)):\n",
    "    plt.text(bar.get_width() + 10, bar.get_y() + bar.get_height()/2, \n",
    "             f'{count:,}', va='center', fontsize=10)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('visuals/top_countries.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "# Print insights\n",
    "print(f\"🌍 Top 5 Countries:\")\n",
    "for i, (country, count) in enumerate(top_countries[:5], 1):\n",
    "    percentage = (count / sum(counts)) * 100\n",
    "    print(f\"   {i}. {country}: {count:,} titles ({percentage:.1f}%)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. 🎭 Top Genres Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze genres\n",
    "genre_counts = Counter()\n",
    "df['listed_in'].dropna().apply(lambda x: genre_counts.update([g.strip() for g in x.split(',')]))\n",
    "top_genres = genre_counts.most_common(15)\n",
    "\n",
    "# Create visualization\n",
    "genres, counts = zip(*top_genres)\n",
    "plt.figure(figsize=(12, 10))\n",
    "bars = plt.barh(range(len(genres)), counts, color=sns.color_palette('plasma', len(genres)))\n",
    "plt.yticks(range(len(genres)), genres)\n",
    "plt.xlabel('Number of Titles', fontsize=12)\n",
    "plt.title('Top 15 Genres on Netflix', fontsize=16, fontweight='bold')\n",
    "\n",
    "# Add value labels\n",
    "for i, (bar, count) in enumerate(zip(bars, counts)):\n",
    "    plt.text(bar.get_width() + 5, bar.get_y() + bar.get_height()/2, \n",
    "             f'{count:,}', va='center', fontsize=10)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('visuals/genre_distribution.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "# Print insights\n",
    "print(f\"🎭 Top 5 Genres:\")\n",
    "for i, (genre, count) in enumerate(top_genres[:5], 1):\n",
    "    percentage = (count / sum(counts)) * 100\n",
    "    print(f\"   {i}. {genre}: {count:,} titles ({percentage:.1f}%)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. 🎬 Top Actors and Directors"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze actors\n",
    "actor_counts = Counter()\n",
    "df['cast'].dropna().apply(lambda x: actor_counts.update([a.strip() for a in x.split(',') if a.strip() != 'Unknown']))\n",
    "top_actors = actor_counts.most_common(10)\n",
    "\n",
    "# Create actors visualization\n",
    "actors, actor_counts_list = zip(*top_actors)\n",
    "plt.figure(figsize=(12, 8))\n",
    "bars = plt.barh(range(len(actors)), actor_counts_list, color=sns.color_palette('coolwarm', len(actors)))\n",
    "plt.yticks(range(len(actors)), actors)\n",
    "plt.xlabel('Number of Appearances', fontsize=12)\n",
    "plt.title('Top 10 Most Featured Actors on Netflix', fontsize=16, fontweight='bold')\n",
    "\n",
    "# Add value labels\n",
    "for i, (bar, count) in enumerate(zip(bars, actor_counts_list)):\n",
    "    plt.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, \n",
    "             f'{count}', va='center', fontsize=10)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('visuals/top_actors.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "# Analyze directors\n",
    "director_counts = Counter()\n",
    "df['director'].dropna().apply(lambda x: director_counts.update([d.strip() for d in x.split(',') if d.strip() != 'Unknown']))\n",
    "top_directors = director_counts.most_common(10)\n",
    "\n",
    "# Create directors visualization\n",
    "directors, director_counts_list = zip(*top_directors)\n",
    "plt.figure(figsize=(12, 8))\n",
    "bars = plt.barh(range(len(directors)), director_counts_list, color=sns.color_palette('viridis', len(directors)))\n",
    "plt.yticks(range(len(directors)), directors)\n",
    "plt.xlabel('Number of Titles Directed', fontsize=12)\n",
    "plt.title('Top 10 Most Prolific Directors on Netflix', fontsize=16, fontweight='bold')\n",
    "\n",
    "# Add value labels\n",
    "for i, (bar, count) in enumerate(zip(bars, director_counts_list)):\n",
    "    plt.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, \n",
    "             f'{count}', va='center', fontsize=10)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('visuals/top_directors.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "# Print insights\n",
    "print(f\"🎬 Top 5 Actors:\")\n",
    "for i, (actor, count) in enumerate(top_actors[:5], 1):\n",
    "    print(f\"   {i}. {actor}: {count} appearances\")\n",
    "\n",
    "print(f\"\\n🎬 Top 5 Directors:\")\n",
    "for i, (director, count) in enumerate(top_directors[:5], 1):\n",
    "    print(f\"   {i}. {director}: {count} titles\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. 🕒 Content Duration Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Separate movies and TV shows\n",
    "movies = df[df['type'] == 'Movie']\n",
    "shows = df[df['type'] == 'TV Show']\n",
    "\n",
    "# Extract duration data\n",
    "movies['minutes'] = movies['duration'].str.extract(r'(\\d+)').astype(float)\n",
    "shows['seasons'] = shows['duration'].str.extract(r'(\\d+)').astype(float)\n",
    "\n",
    "# Create visualization\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))\n",
    "\n",
    "# Movie duration distribution\n",
    "ax1.hist(movies['minutes'].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')\n",
    "ax1.set_title('Movie Duration Distribution', fontsize=14, fontweight='bold')\n",
    "ax1.set_xlabel('Duration (minutes)')\n",
    "ax1.set_ylabel('Number of Movies')\n",
    "ax1.axvline(movies['minutes'].mean(), color='red', linestyle='--', \n",
    "            label=f'Mean: {movies[\"minutes\"].mean():.1f} min')\n",
    "ax1.legend()\n",
    "\n",
    "# TV show seasons distribution\n",
    "ax2.hist(shows['seasons'].dropna(), bins=20, alpha=0.7, color='lightcoral', edgecolor='black')\n",
    "ax2.set_title('TV Show Seasons Distribution', fontsize=14, fontweight='bold')\n",
    "ax2.set_xlabel('Number of Seasons')\n",
    "ax2.set_ylabel('Number of TV Shows')\n",
    "ax2.axvline(shows['seasons'].mean(), color='red', linestyle='--', \n",
    "            label=f'Mean: {shows[\"seasons\"].mean():.1f} seasons')\n",
    "ax2.legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('visuals/duration_analysis.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "# Print insights\n",
    "print(f\"📊 Duration Insights:\")\n",
    "print(f\"   • Average movie duration: {movies['minutes'].mean():.1f} minutes\")\n",
    "print(f\"   • Average TV show seasons: {shows['seasons'].mean():.1f} seasons\")\n",
    "print(f\"   • Longest movie: {movies['minutes'].max():.0f} minutes\")\n",
    "print(f\"   • Most seasons in a show: {shows['seasons'].max():.0f} seasons\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. 📅 Temporal Trends Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create heatmap of content addition by year and month\n",
    "trend_data = df.groupby(['year_added', 'month_added']).size().unstack(fill_value=0)\n",
    "\n",
    "plt.figure(figsize=(14, 8))\n",
    "sns.heatmap(trend_data, cmap='YlGnBu', annot=True, fmt='d', cbar_kws={'label': 'Number of Titles'})\n",
    "plt.title('Content Addition Patterns by Year and Month', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Month', fontsize=12)\n",
    "plt.ylabel('Year', fontsize=12)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('visuals/temporal_trends.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "# Monthly analysis\n",
    "monthly_trends = df.groupby('month_added').size()\n",
    "month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', \n",
    "               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']\n",
    "\n",
    "plt.figure(figsize=(12, 6))\n",
    "plt.bar(range(1, 13), monthly_trends.values, color=sns.color_palette('Set3', 12))\n",
    "plt.title('Content Addition by Month', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Month', fontsize=12)\n",
    "plt.ylabel('Number of Titles', fontsize=12)\n",
    "plt.xticks(range(1, 13), month_names)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('visuals/monthly_trends.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "# Print insights\n",
    "peak_month = monthly_trends.idxmax()\n",
    "print(f\"📅 Temporal Insights:\")\n",
    "print(f\"   • Peak month for content addition: {month_names[peak_month-1]} ({monthly_trends[peak_month]} titles)\")\n",
    "print(f\"   • Slowest month: {month_names[monthly_trends.idxmin()-1]} ({monthly_trends.min()} titles)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. 🔞 Content Ratings Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze ratings\n",
    "rating_counts = df['rating'].value_counts().head(10)\n",
    "\n",
    "# Create visualization\n",
    "plt.figure(figsize=(12, 8))\n",
    "bars = plt.barh(range(len(rating_counts)), rating_counts.values, \n",
    "                color=sns.color_palette('RdYlBu', len(rating_counts)))\n",
    "plt.yticks(range(len(rating_counts)), rating_counts.index)\n",
    "plt.xlabel('Number of Titles', fontsize=12)\n",
    "plt.title('Top 10 Content Ratings on Netflix', fontsize=16, fontweight='bold')\n",
    "\n",
    "# Add value labels\n",
    "for i, (bar, count) in enumerate(zip(bars, rating_counts.values)):\n",
    "    percentage = (count / len(df)) * 100\n",
    "    plt.text(bar.get_width() + 10, bar.get_y() + bar.get_height()/2, \n",
    "             f'{count:,} ({percentage:.1f}%)', va='center', fontsize=10)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('visuals/rating_distribution.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "# Print insights\n",
    "print(f\"🔞 Rating Insights:\")\n",
    "for i, (rating, count) in enumerate(rating_counts.items(), 1):\n",
    "    percentage = (count / len(df)) * 100\n",
    "    print(f\"   {i}. {rating}: {count:,} titles ({percentage:.1f}%)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. 📊 Summary Statistics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate comprehensive summary\n",
    "print(\"📊 NETFLIX CONTENT ANALYSIS SUMMARY\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "print(f\"\\n📈 Content Overview:\")\n",
    "print(f\"   • Total titles: {len(df):,}\")\n",
    "print(f\"   • Movies: {len(df[df['type'] == 'Movie']):,} ({len(df[df['type'] == 'Movie'])/len(df)*100:.1f}%)\")\n",
    "print(f\"   • TV Shows: {len(df[df['type'] == 'TV Show']):,} ({len(df[df['type'] == 'TV Show'])/len(df)*100:.1f}%)\")\n",
    "print(f\"   • Date range: {df['release_year'].min()} - {df['release_year'].max()}\")\n",
    "\n",
    "print(f\"\\n🌍 Geographic Distribution:\")\n",
    "print(f\"   • Countries represented: {len(country_counts)}\")\n",
    "print(f\"   • Top country: {top_countries[0][0]} ({top_countries[0][1]:,} titles)\")\n",
    "\n",
    "print(f\"\\n🎭 Genre Diversity:\")\n",
    "print(f\"   • Unique genres: {len(genre_counts)}\")\n",
    "print(f\"   • Most popular: {top_genres[0][0]} ({top_genres[0][1]:,} titles)\")\n",
    "\n",
    "print(f\"\\n🎬 Talent Pool:\")\n",
    "print(f\"   • Unique actors: {len(actor_counts)}\")\n",
    "print(f\"   • Unique directors: {len(director_counts)}\")\n",
    "print(f\"   • Most featured actor: {top_actors[0][0]} ({top_actors[0][1]} appearances)\")\n",
    "print(f\"   • Most prolific director: {top_directors[0][0]} ({top_directors[0][1]} titles)\")\n",
    "\n",
    "print(f\"\\n📅 Temporal Patterns:\")\n",
    "print(f\"   • Peak addition year: {int(max_year['year_added'])} ({max_year['count']} titles)\")\n",
    "print(f\"   • Peak month: {month_names[peak_month-1]} ({monthly_trends[peak_month]} titles)\")\n",
    "\n",
    "print(f\"\\n🔞 Content Ratings:\")\n",
    "print(f\"   • Most common rating: {rating_counts.index[0]} ({rating_counts.iloc[0]:,} titles)\")\n",
    "print(f\"   • Mature content (TV-MA): {rating_counts.get('TV-MA', 0):,} titles\")\n",
    "\n",
    "print(f\"\\n✅ Analysis completed! All visualizations saved to 'visuals/' folder.\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.8"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}